# Step 06 - Set Up Advanced Model Monitoring

Use [Azure ML's built-in model/data drift monitoring](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-monitor-model-performance?view=azureml-api-2&tabs=python) using the advanced mode which allows for comparison of incoming inferencing data relative to baseline data used for training data. 

In [ ]:
from azure.ai.ml import MLClient, Input
from azure.ai.ml.entities import Data, ManagedOnlineEndpoint, ManagedOnlineDeployment, Model, Environment, ModelPackage, BaseEnvironment, AzureMLOnlineInferencingServer, CodeConfiguration, DataCollector, DeploymentCollection
from azure.ai.ml.constants import AssetTypes
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import ContainerRegistryCredential

from azure.identity import DefaultAzureCredential
from azure.ai.ml import Input, MLClient
from azure.ai.ml.constants import (
    MonitorDatasetContext,
)
from azure.ai.ml.entities import (
    AlertNotification,
    BaselineDataRange,
    DataDriftSignal,
    DataQualitySignal,
    PredictionDriftSignal,
    DataDriftMetricThreshold,
    DataQualityMetricThreshold,
    FeatureAttributionDriftMetricThreshold,
    FeatureAttributionDriftSignal,
    PredictionDriftMetricThreshold,
    NumericalDriftMetrics,
    CategoricalDriftMetrics,
    DataQualityMetricsNumerical,
    DataQualityMetricsCategorical,
    MonitorFeatureFilter,
    MonitoringTarget,
    MonitorDefinition,
    MonitorSchedule,
    RecurrencePattern,
    RecurrenceTrigger,
    ServerlessSparkCompute,
    ReferenceData,
    ProductionData
)



subscription_id = "..."
resource_group = "..."
workspace = "..."
model_name = "zone1-power-consumption-xgboost-model"

ml_client = MLClient(DefaultAzureCredential(), subscription_id, resource_group, workspace)
ml_client

### Register training data from ADLS Gen2 Delta as a data asset in Azure ML

Registration below demonstrates how to retrieve data from ADLS Gen2 delta tables as a specified timepoint

In [ ]:
# Save and register dataset in AML workspace
import pandas as pd
from mltable import from_delta_lake
from datetime import datetime, timezone
mltable_ts = from_delta_lake(delta_table_uri="azureml://subscriptions/<SUB>/resourcegroups/<RG>/workspaces/<WORKSPACE>/datastores/<DATASOURCE>/paths/train/", timestamp_as_of=datetime.now(timezone.utc).isoformat())
mltable_ts

mltable_folder = "./traindata"
mltable_ts.save(mltable_folder)

my_data = Data(
    path=mltable_folder,
    type=AssetTypes.MLTABLE,
    description="Power consumption training dataset",
    name="power_consumption_train",
    version='1',
)
ml_client.data.create_or_update(my_data)

### Configure advanced montoring

Sample below showcases how to configure feature-level monitoring and drift comparison relative to source training data

In [10]:
# create your compute
spark_compute = ServerlessSparkCompute(
    instance_type="standard_e4s_v3",
    runtime_version="3.3"
)

# specify the online deployment (if you have one)
monitoring_target = MonitoringTarget(
    ml_task="regression",
    endpoint_deployment_id="azureml:power-endpoint:blue"
)

# specify a lookback window size and offset, or omit this to use the defaults, which are specified in the documentation
data_window = BaselineDataRange(lookback_window_size="P1D", lookback_window_offset="P0D")

production_data = ProductionData(
    input_data=Input(
        type="uri_folder",
        path="azureml:power-endpoint-blue-model_inputs:1"
    ),
    data_window=data_window,
    data_context=MonitorDatasetContext.MODEL_INPUTS,
)

# training data to be used as reference dataset
reference_data_training = ReferenceData(
    input_data=Input(
        type="mltable",
        path="azureml:power_consumption_train:1"
    ),
    data_column_names={
        "target_column":"Zone-1-Power-Consumption"
    },
    data_context=MonitorDatasetContext.TRAINING,
)

# create an advanced data drift signal
features = MonitorFeatureFilter(top_n_feature_importance=10)

metric_thresholds = DataDriftMetricThreshold(
    numerical=NumericalDriftMetrics(
        jensen_shannon_distance=0.01
    ),
    categorical=CategoricalDriftMetrics(
        pearsons_chi_squared_test=0.02
    )
)

advanced_data_drift = DataDriftSignal(
    reference_data=reference_data_training,
    features=features,
    metric_thresholds=metric_thresholds
)

# create an advanced prediction drift signal
metric_thresholds = PredictionDriftMetricThreshold(
    categorical=CategoricalDriftMetrics(
        jensen_shannon_distance=0.01
    )
)

advanced_prediction_drift = PredictionDriftSignal(
    reference_data=reference_data_training,
    metric_thresholds=metric_thresholds
)

# create an advanced data quality signal
features = ['Temperature', 'Humidity', 'Wind-Speed', 'general-diffuse-flows', 'diffuse-flows']

metric_thresholds = DataQualityMetricThreshold(
    numerical=DataQualityMetricsNumerical(
        null_value_rate=0.01
    ),
    categorical=DataQualityMetricsCategorical(
        out_of_bounds_rate=0.02
    )
)

advanced_data_quality = DataQualitySignal(
    reference_data=reference_data_training,
    features=features,
    metric_thresholds=metric_thresholds,
    alert_enabled=False
)

# create feature attribution drift signal
metric_thresholds = FeatureAttributionDriftMetricThreshold(normalized_discounted_cumulative_gain=0.9)

feature_attribution_drift = FeatureAttributionDriftSignal(
    reference_data=reference_data_training,
    metric_thresholds=metric_thresholds,
    alert_enabled=False
)

# put all monitoring signals in a dictionary
monitoring_signals = {
    'data_drift_advanced':advanced_data_drift,
    'data_quality_advanced':advanced_data_quality,
    'feature_attribution_drift':feature_attribution_drift,
}

# create alert notification object

# create the monitor definition
monitor_definition = MonitorDefinition(
    compute=spark_compute,
    monitoring_target=monitoring_target,
    monitoring_signals=monitoring_signals,
    #alert_notification=alert_notification
)

# specify the frequency on which to run your monitor
recurrence_trigger = RecurrenceTrigger(
    frequency="day",
    interval=1,
    schedule=RecurrencePattern(hours=3, minutes=15)
)

# create your monitor
model_monitor = MonitorSchedule(
    name="power_endpoint_advanced",
    trigger=recurrence_trigger,
    create_monitor=monitor_definition
)

poller = ml_client.schedules.begin_create_or_update(model_monitor)
created_monitor = poller.result()

Class FeatureAttributionDriftMetricThreshold: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class FeatureAttributionDriftSignal: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class FADProductionData: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


...